In [1]:
import pandas as pd
import yfinance as yf
from datetime import datetime

start_date = datetime.now() - pd.DateOffset(months=3)
end_date = datetime.now()

tickers = ['AAPL', 'MSFT', 'NFLX', 'GOOG']

df_list = []

for ticker in tickers:
    data = yf.download(ticker, start=start_date, end=end_date)
    df_list.append(data)

df = pd.concat(df_list, keys=tickers, names=['Ticker', 'Date'])
print(df.head())

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
                         Open        High         Low       Close   Adj Close  \
Ticker Date                                                                     
AAPL   2023-07-03  193.779999  193.880005  191.759995  192.460007  192.200470   
       2023-07-05  191.570007  192.979996  190.619995  191.330002  191.071976   
       2023-07-06  189.839996  192.020004  189.199997  191.809998  191.551331   
       2023-07-07  191.410004  192.669998  190.240005  190.679993  190.422852   
       2023-07-10  189.259995  189.990005  187.039993  188.610001  188.355652   

                     Volume  
Ticker Date                  
AAPL   2023-07-03  31458200  
       2023-07-05  46920300  
       2023-07-06  45094300  
      

In [2]:
df = df.reset_index()
print(df.head())

  Ticker       Date        Open        High         Low       Close  \
0   AAPL 2023-07-03  193.779999  193.880005  191.759995  192.460007   
1   AAPL 2023-07-05  191.570007  192.979996  190.619995  191.330002   
2   AAPL 2023-07-06  189.839996  192.020004  189.199997  191.809998   
3   AAPL 2023-07-07  191.410004  192.669998  190.240005  190.679993   
4   AAPL 2023-07-10  189.259995  189.990005  187.039993  188.610001   

    Adj Close    Volume  
0  192.200470  31458200  
1  191.071976  46920300  
2  191.551331  45094300  
3  190.422852  46778000  
4  188.355652  59922200  


In [3]:
import plotly.express as px
fig = px.line(df, x='Date',
              y='Close',
              color='Ticker',
              title="Stock Market Performance for the Last 3 Months")
fig.show()

In [4]:
fig = px.area(df, x='Date', y='Close', color='Ticker',
              facet_col='Ticker',
              labels={'Date':'Date', 'Close':'Closing Price', 'Ticker':'Company'},
              title='Stock Prices for Apple, Microsoft, Netflix, and Google')
fig.show()

In [5]:
df['MA10'] = df.groupby('Ticker')['Close'].rolling(window=10).mean().reset_index(0, drop=True)
df['MA20'] = df.groupby('Ticker')['Close'].rolling(window=20).mean().reset_index(0, drop=True)

for ticker, group in df.groupby('Ticker'):
    print(f'Moving Averages for {ticker}')
    print(group[['MA10', 'MA20']])

Moving Averages for AAPL
          MA10        MA20
0          NaN         NaN
1          NaN         NaN
2          NaN         NaN
3          NaN         NaN
4          NaN         NaN
..         ...         ...
59  175.425002  179.368000
60  175.047000  178.683500
61  174.542000  177.835500
62  174.162001  177.002501
63  173.740001  176.217001

[64 rows x 2 columns]
Moving Averages for GOOG
           MA10        MA20
192         NaN         NaN
193         NaN         NaN
194         NaN         NaN
195         NaN         NaN
196         NaN         NaN
..          ...         ...
251  135.140001  135.863001
252  134.536002  135.661501
253  133.950002  135.471502
254  133.305002  135.196502
255  132.926001  135.115002

[64 rows x 2 columns]
Moving Averages for MSFT
           MA10        MA20
64          NaN         NaN
65          NaN         NaN
66          NaN         NaN
67          NaN         NaN
68          NaN         NaN
..          ...         ...
123  324.968002  328.18

In [6]:
for ticker, group in df.groupby('Ticker'):
    fig = px.line(group, x='Date', y=['Close', 'MA10', 'MA20'],
                  title=f"{ticker} Moving Averages")
    fig.show()

In [7]:
df['Volatility'] = df.groupby('Ticker')['Close'].pct_change().rolling(window=10).std().reset_index(0, drop=True)
fig = px.line(df, x='Date', y='Volatility',
              color='Ticker',
              title='Volatility of All Companies')
fig.show()

In [8]:
# create a DataFrame with the stock prices of Apple and Microsoft
apple = df.loc[df['Ticker'] == 'AAPL', ['Date', 'Close']].rename(columns={'Close': 'AAPL'})
microsoft = df.loc[df['Ticker'] == 'MSFT', ['Date', 'Close']].rename(columns={'Close': 'MSFT'})
df_corr = pd.merge(apple, microsoft, on='Date')

# create a scatter plot to visualize the correlation
fig = px.scatter(df_corr, x='AAPL', y='MSFT',
                 trendline='ols',
                 title='Correlation between Apple and Microsoft')
fig.show()

In [9]:
# create a DataFrame with the stock prices of Google and Netflix
google = df.loc[df['Ticker'] == 'GOOG', ['Date', 'Close']].rename(columns={'Close': 'GOOG'})
netflix = df.loc[df['Ticker'] == 'NFLX', ['Date', 'Close']].rename(columns={'Close': 'NFLX'})
df_corr = pd.merge(google, netflix, on='Date')

# create a scatter plot to visualize the correlation
fig = px.scatter(df_corr, x='GOOG', y='NFLX',
                 trendline='ols',
                 title='Correlation between Google and Netflix')
fig.show()